In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from util import gloveutil, const

In [3]:
fm = const.FileManager()
glove_200 = gloveutil.load_glove(fm.GLOVE_200)

In [4]:
from util import reader
df = reader.load_df()

In [5]:
maxlen = 30
nwords = 10000
embedding_dim = 200

In [6]:
from sklearn.model_selection import train_test_split
import keras.preprocessing.text as Ktext
import keras.preprocessing.sequence as Kseq

xs = df.body_content
tokenizer = Ktext.Tokenizer(num_words=nwords)
tokenizer.fit_on_texts(xs)
xs_seq = Kseq.pad_sequences(
    tokenizer.texts_to_sequences(xs), maxlen=maxlen)

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [7]:
import numpy as np
model_2_dim = 20

x_train = xs_seq
x_train_2 = np.zeros((xs_seq.shape[0], model_2_dim))
y_train = df.label.values

In [8]:
df.body_atreplies = df.body_atreplies.apply(lambda r: r if isinstance(r, list) else [])

In [9]:
df[[type(r) != list for r in df.body_atreplies]]

,children,created_at,favorite_count,id,label,media,parents,reply_count,retweet_count,user_handle,user_id,user_is_verified,user_name,body_atreplies,body_content,body_hashtags,body_links


In [10]:
user_index = dict()
i = 0
for atreplies in df.body_atreplies:
    for atreply in filter(lambda x: x not in user_index, atreplies):
        user_index[atreply] = i
        i += 1
len(user_index)

26324

In [11]:
users_matrix = np.zeros((len(df), len(user_index)))
for row, v in enumerate(df.body_atreplies.head()):
    for user in v:
        users_matrix[row, user_index[user]] = 1

In [16]:
print(xs.iloc[0])
print(xs_seq[0])
print([k for k, v in tokenizer.word_index.items() if v == 96])
print(users_matrix.shape)

@business A Korean-American man detained in North Korea has reportedly confessed to trying... http://goo.gl/fb/0KGMKc 
[   0    0    0    0    0    0    0    0    0    0    0    0    0   96
    9 2532  314  186 3320    6  424  664   68 2197    4  843    2  255
  256  178]
['business']
(101338, 26324)


In [18]:
from keras.layers import Input, Embedding, Layer
from keras.models import Model

embedding_matrix = gloveutil.glove_to_embedding_matrix(
    glove_200, tokenizer.word_index)
X = Input(shape=(maxlen, ))
embed = Embedding(
  len(tokenizer.word_index), embedding_dim, weights=[embedding_matrix],
  input_length=maxlen, trainable=False)

In [24]:
import keras.layers as Klay
import keras.regularizers as Kreg
from keras import Sequential

model_seq = Sequential([
    embed,
    #Klay.Conv1D(filters=100, kernel_size=5, padding='same'),
    #Klay.MaxPool1D(pool_size=2),
    #Klay.Conv1D(filters=100, kernel_size=5, padding='same'),
    Klay.LSTM(100),
    #Klay.Flatten(),
    Klay.Dense(100, activation='relu'),
])

model_seq2 = Sequential([
    Klay.Dense(100, input_shape=(users_matrix.shape[1],)),
    Klay.Dense(100)
])

concat = Klay.concatenate([model_seq.output, model_seq2.output])
Y = Klay.Dense(4, activation='sigmoid')(concat)
model_combined = Model([model_seq.input, model_seq2.input], Y)

In [25]:
model_combined.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['f1', 'f1_macro'])
model_combined.fit(
    [x_train, users_matrix], 
    y_train,
    batch_size=128,
    epochs=10,
    validation_split=0.2)

Train on 81070 samples, validate on 20268 samples
Epoch 1/10
81070/81070 [==============================] - 159s 2ms/step - loss: 0.2290 - acc: 0.9032 - val_loss: 1.1961 - val_acc: 0.4883
Epoch 2/10
81070/81070 [==============================] - 170s 2ms/step - loss: 0.1488 - acc: 0.9404 - val_loss: 1.1515 - val_acc: 0.6000
Epoch 3/10
81070/81070 [==============================] - 152s 2ms/step - loss: 0.1287 - acc: 0.9483 - val_loss: 1.2875 - val_acc: 0.5633
Epoch 4/10
81070/81070 [==============================] - 167s 2ms/step - loss: 0.1137 - acc: 0.9526 - val_loss: 1.4613 - val_acc: 0.5237
Epoch 5/10
81070/81070 [==============================] - 173s 2ms/step - loss: 0.1008 - acc: 0.9583 - val_loss: 1.5962 - val_acc: 0.6129
Epoch 6/10
81070/81070 [==============================] - 166s 2ms/step - loss: 0.0893 - acc: 0.9627 - val_loss: 1.9920 - val_acc: 0.5694
Epoch 7/10
81070/81070 [==============================] - 176s 2ms/step - loss: 0.0775 - acc: 0.9674 - val_loss: 2.2371 - 

In [32]:
from keras.layers import Dense, Flatten
X_embed = embed(X)
Y = Flatten()(X_embed)
Y = Dense(1)(Y)
model = Model(X, Y)

print(X_embed)

In [35]:

model.compile(
    loss='binary_crossentropy',
    optimizer='rmsprop',
    metrics=['acc'])

model.fit(
    x_train, y_train,
    batch_size=128,
    epochs=10,
    validation_split=0.2)


Epoch 1/10
101338/101338 [==============================] - 3s 26us/step - loss: 0.8911 - acc: 0.6429
Epoch 2/10
101338/101338 [==============================] - 2s 23us/step - loss: 0.5846 - acc: 0.6867
Epoch 3/10
101338/101338 [==============================] - 2s 23us/step - loss: 0.5523 - acc: 0.6772
Epoch 4/10
101338/101338 [==============================] - 2s 24us/step - loss: 0.5529 - acc: 0.6574
Epoch 5/10
101338/101338 [==============================] - 2s 24us/step - loss: 0.5237 - acc: 0.6567
Epoch 6/10
101338/101338 [==============================] - 2s 24us/step - loss: 0.5348 - acc: 0.6477
Epoch 7/10
101338/101338 [==============================] - 3s 29us/step - loss: 0.5120 - acc: 0.6521
Epoch 8/10
101338/101338 [==============================] - 3s 29us/step - loss: 0.5169 - acc: 0.6413
Epoch 9/10
101338/101338 [==============================] - 3s 26us/step - loss: 0.5098 - acc: 0.6366
Epoch 10/10
101338/101338 [==============================] - 3s 27us/step - loss: 

In [31]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 20)                0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 20, 200)           39630000  
_________________________________________________________________
dense_2 (Dense)              (None, 20, 1)             201       
Total params: 39,630,201
Trainable params: 201
Non-trainable params: 39,630,000
_________________________________________________________________
